### Builtins

In [1]:
import os
import re
import json
import traceback

### Common Libs

In [2]:
def import_object(name):
    """
    Imports an object by name.

    import_object('x') is equivalent to 'import x'.
    import_object('x.y.z') is equivalent to 'from x.y import z'.
    
    """
    if name.count('.') == 0:
        return __import__(name, None, None)

    parts = name.split('.')
    obj = __import__('.'.join(parts[:-1]), None, None, [parts[-1]], 0)
    try:
        return getattr(obj, parts[-1])
    except AttributeError:
        raise ImportError("No module named %s" % parts[-1])

In [3]:
def get_real(value):
    
    value_str = str(value)
    
    if value_str == "None":
        return None
    
    try:
        return int(value_str)
    except:
        pass

    try:
        return float(value_str)
    except:
        pass

    try:
        if value_str.lower() == "true":
            return True
        elif value_str.lower() == "false":
            return False
    except:
        pass

    if value_str.startswith("'") and value_str.endswith("'"):
        return value_str[1:-1]

    return value

In [4]:
def get_params(raw_params):
    # raw_params = ["1", "2.36", "x", "y", "a =1", "  b= '2'", "c=true  ", "    d =  'z'        ", "e=1.589"]
    # return:
    # (args, kwargs)
    # args = [1,2.36,"x","y"]
    # kwargs = {"a":1, "b": '2', "c": True, "d": 'z', "e": 1.589}
    assert isinstance(raw_params, (list, tuple)), "parameter `raw_params` shoule be a list or tuple"
    args, kwargs = [], {}
    for raw_param in raw_params:
        formated_raw_param = re.sub(r"\s*=\s*", "=", raw_param.strip())
        kwarg_matched = re.search(r"\s*(.*)\s*=\s*(.*)\s*", formated_raw_param)
        if kwarg_matched:
            key, value = kwarg_matched.groups()
            # value can be int, float, str, bool
            kwargs[key] = get_real(value)
        else:
            args.append(get_real(raw_param))
    return args, kwargs

In [5]:
class JsonExtHook:

    def __init__(self, ref_dir=None):
        self.ref_dir = ref_dir or ""

    def __call__(self, item):
        for key, value in item.items():  # 如需计算, 不允许嵌套, 即 计算所需参数 就是静态值, 而不需要再计算
            if not isinstance(value, str):
                item[key] = value

            elif value.startswith("\\"):
                value = re.sub(r"^\\", "", value).strip()

            elif value.startswith("call:"):
                value = re.sub(r"^call:", "", value).strip()

                need_compute = re.search(r"([^()]*)\((.*)\)", str(value))

                if need_compute:
                    func_module, raw_params = need_compute.groups()
                    args, kwargs = get_params(raw_params.split(","))
                    try:
                        func = import_object(func_module)
                        value = func(*args, **kwargs)
                    except Exception as E:
                        print_error("Error while hooking <{k}:{v}>: {e}\n"
                                    "\tthis item remains not hooked".format(k=key, v=value, e=E))

            elif value.startswith("file:"):
                file_path = re.sub(r"^file:", "", value).strip()
                file_path = file_path if os.path.exists(file_path) else os.path.join(self.ref_dir, file_path)
                try:
                    with open(file_path, "r", encoding="utf-8") as f:
                        value = f.read()
                except:
                    pass

            item[key] = value

        return item

### Read Json Without Comments

In [6]:
def read_json(jsonfile, hook=False, encoding="utf-8"):
    """
        从文件中读取一个json文件
        如果指定hook=True:
            则当值为"file:filename.txt"时，能自动将filename.txt的值读入以代替原值
            则当值为"call:voerka.aaa.aaa(1,2,3)"时，能自动导入该模块函数并执行，将执行结果读入以代替原值

    """
    content = ""
    try:
        with open(jsonfile, "r") as fs:
            object_hook = JsonExtHook(ref_dir=os.path.dirname(jsonfile)) if hook else None
            content = json.loads(fs.read(), object_hook=object_hook, encoding=encoding)
    except Exception as E:
        raise Exception("Error while reading json from %s: %s" % (jsonfile, E))

    return content

### Test: Read Json Without Comments

In [7]:
JSON_FILE = "test.json"
result = read_json(JSON_FILE,  hook=True)

In [8]:
print("result: %s" % result)

result: {'b': 2, 'd': 'test.json', 'a': datetime.date(1999, 9, 11), 'e': None, 'c': 'true'}


### Read Json With Comments

In [9]:
def find_line_comment(line):
    """
    find the comment within a line
    :param line:
    :return:
    """
    splited_line = line.split("\\\\")
    quotation_count = 0
    splited_line_len = len(splited_line)
    if splited_line_len:
        for index, element in enumerate(splited_line):
            quotation_count += element.count('"')
            if quotation_count % 2 == 0:
                return ("\\\\" + "\\\\".join(splited_line[(index+1):])) if index < splited_line_len - 1 else ""
    return ""


def read_json_with_comments(jsonfile, hook=False, encoding="utf-8"):
    """
    read a json file with comments and correctly return json
    :param jsonfile:
    :return:
    """
    fc = ""
    try:
        with open(jsonfile, "r", encoding=encoding) as fs:
            for line in fs.readlines():
                new_line = line.replace(find_line_comment(line), "")
                fc += "\n" + new_line
            object_hook = JsonExtHook(ref_dir=os.path.dirname(jsonfile)) if hook else None
            if fc.startswith(u'\ufeff'):
                fc = fc.encode('utf8')[3:].decode('utf8')  # remove Unexpected UTF-8 BOM (decode using utf-8-sig)
            content = json.loads(fc, object_hook=object_hook, encoding=encoding)
            return content
    except Exception as E:
        raise Exception("Error while reading json from %s: %s" % (jsonfile, E))

### Test: Read Json With Comments

In [10]:
JSON_FILE_WITH_COMMENTS = "test_Remove_Comments_in_JSON.json"
result = read_json_with_comments(JSON_FILE_WITH_COMMENTS, hook=True)
print("result: %s\n" % json.dumps(result, indent=4))

result: {
    "b": "1",
    "a": 0,
    "e": "",
    "c": " \\",
    "d": " http:\\",
    "email": "http:\\admin@qq.com"
}

